In [10]:
import librosa
import numpy as np
from model import simple_model_lung

In [11]:
model = simple_model_lung()

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.load_weights('../result/lung_model.ckpt')

In [14]:
def audio_features(filename):
    sound, sample_rate = librosa.load(filename)
    stft = np.abs(librosa.stft(sound)) # short-time Fourier transform (STFT) spectrogram
    
    mfccs = np.mean(librosa.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40), axis=1) # Mel-frequency cepstral coefficients
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate), axis=1) # Chromagram from STFT spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=sound, sr=sample_rate), axis=1) # Mel-scaled spectrogram
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate), axis=1) # Spectral contrast
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound), sr=sample_rate), axis=1) # Tonal centroid features
    
    features = np.concatenate((mfccs, chroma, mel, contrast, tonnetz))
    return features

In [15]:
def predict_sound(sound_file):
    classes = ["COPD", "Healthy", "URTI", "Bronchiectasis", "Pneumonia", "Bronchiolitis"]
    data = audio_features(sound_file)
    data = np.reshape(data, (1, data.shape[0], 1))
    preds = model.predict(data, verbose=0)
    classpreds = np.argmax(preds, axis=1)[0]
    print(classes[classpreds])
    
predict_sound("C:\lung_sound\dataset\sound_txt\\169_1b2_Ll_sc_Meditron.wav")

Bronchiectasis


In [16]:
predict_sound("C:\lung_sound\dataset\sound_txt\\224_1b2_Al_sc_Meditron.wav")

Healthy


In [17]:
predict_sound("C:\lung_sound\dataset\sound_txt\\172_1b4_Pl_mc_AKGC417L.wav")

COPD


In [20]:
predict_sound("C:\lung_sound\dataset\sound_txt\\173_1b1_Al_sc_Meditron.wav")

Bronchiolitis
